In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [28]:
data = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv')
test= pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv')

data.columns = data.columns.str.strip()
test.columns = test.columns.str.strip()

data.columns


Index(['id', 'Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Genre', 'Host_Popularity_percentage', 'Publication_Day',
       'Publication_Time', 'Guest_Popularity_percentage', 'Number_of_Ads',
       'Episode_Sentiment', 'Listening_Time_minutes'],
      dtype='object')

In [29]:
def clean(data):
    data.columns = data.columns.str.strip()  # Remove leading/trailing spaces
    print("Columns in DataFrame:", data.columns.tolist())  # Debug: show columns
    
    # List of columns to drop
    columns_to_drop = ['Podcast_Name','id','Number_of_Ads','Episode_Title', 'Genre', 'Publication_Time', 'Episode_Sentiment', 'Publication_Day']
    
    # Only drop columns that exist
    columns_to_drop = [col for col in columns_to_drop if col in data.columns]
    if not columns_to_drop:
        print("Warning: None of the specified columns to drop were found.")
    else:
        data = data.drop(columns_to_drop, axis=1)
    
    # Fill missing values in specified columns
    cols = ["Episode_Length_minutes", "Guest_Popularity_percentage", "Host_Popularity_percentage","Listening_Time_minutes"]
    for col in cols:
        if col in data.columns:
            data[col] = data[col].fillna(data[col].mean())
        else:
            print(f"Warning: Column {col} not found for filling missing values.")
    
    return data

data = clean(data)
test_data = clean(test)
data.columns = data.columns.str.strip() 
data.head(10)
#test.head(10)

Columns in DataFrame: ['id', 'Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre', 'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment', 'Listening_Time_minutes']
Columns in DataFrame: ['id', 'Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre', 'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment']


,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Listening_Time_minutes
0,64.504738,74.81,52.236449,31.41998
1,119.800000,66.95,75.950000,88.01241
2,73.900000,69.97,8.970000,44.92531
3,67.170000,57.22,78.700000,46.27824
4,110.510000,80.07,58.680000,75.61031
5,26.540000,48.96,52.236449,22.77047
6,69.830000,35.82,39.020000,64.75024
7,48.520000,44.99,20.120000,22.37517
8,105.870000,69.81,52.236449,68.00124
9,64.504738,82.18,59.720000,45.94761


In [30]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# Initialize
label_encoder = LabelEncoder()

# Separate features and target
y = data["Listening_Time_minutes"]
x = data.drop("Listening_Time_minutes", axis=1)

# Handle missing values
x = x.fillna(x.mean())
y = y.fillna(y.mean())

# Encode categorical features
for col in x.columns:
    if x[col].dtype == 'object':
        x[col] = label_encoder.fit_transform(x[col])

# Train model
regressor = RandomForestRegressor(n_estimators=250, random_state=20, oob_score=True)
regressor.fit(x, y)




RandomForestRegressor(n_estimators=250, oob_score=True, random_state=20)

In [31]:
from sklearn.metrics import mean_squared_error, r2_score 

oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')

predictions = regressor.predict(x)

mse = mean_squared_error(y, predictions)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y, predictions)
print(f'R-squared: {r2}')


rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')



Out-of-Bag Score: 0.7561999919987111
Mean Squared Error: 33.66402982546557
R-squared: 0.9542910529169796
Root Mean Squared Error: 5.8020711668735645


In [33]:
test_data = test_data[["Episode_Length_minutes","Host_Popularity_percentage","Guest_Popularity_percentage"]]

In [34]:
# Ensure the test data has same columns as train
test_data = test_data[x.columns]

# Now predict
submission_preds = regressor.predict(test_data)


In [38]:
df = pd.DataFrame({"id":test["id"].values,
                 "Listening_time":submission_preds,
                  })

In [39]:
df.to_csv('submission.csv', index=False)